In [1]:
# Connect to gdrive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# **Import**

In [2]:
# Import Library
import os
import sys
import shutil
from multiprocessing import Pool

import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns

import librosa
import soundfile as sf
from scipy.signal import lfilter
from copy import deepcopy
from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub

sys.path.append('/content/drive/My Drive/[3] Model ML TBCare/')
import speechproc

# **Metadata Exploration**

## **Load Metadata**

In [ ]:
df = pd.read_csv("/content/drive/My Drive/[3] Model ML TBCare/Dataset Google + CIDRZ Health AI Evaluation Zambia/Metadata and Codebook/GHAI_Final_Data_2023.csv")
df

,sex,consent_obtained,barcode,coughdur,cough_productive,haemoptysis,chestpain,shortbreath,fever,ngtsweats,...,tb_class_index,tb_predictions,tb_predictions1,age,meters_height,bmi,smear_all,type_tb,ground_truth_tb,facility_code
0,m,1,01-399-0258,1-2wks,yes,no,yes,yes,no,yes,...,1,0.654549,0.345451,31,1.62,24.0,neg,No TBdx,neg,Kan
1,m,1,01-399-1081,<1wk,yes,no,yes,yes,yes,yes,...,1,0.438112,0.561888,58,1.64,14.0,neg,No TBdx,neg,Kan
2,m,1,02-399-0856,<1wk,yes,yes,yes,yes,yes,yes,...,1,0.414009,0.585991,24,1.83,16.0,neg,No TBdx,neg,Cha
3,f,1,03-399-0290,1-2wks,yes,no,yes,no,no,yes,...,1,0.607053,0.392947,25,1.70,21.0,neg,No TBdx,neg,Chai
4,m,1,01-399-0744,<1wk,yes,yes,yes,yes,yes,yes,...,1,0.401624,0.598376,30,1.70,20.0,neg,No TBdx,neg,Kan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,f,1,02-399-0239,no cough,NaN,NaN,no,no,no,no,...,1,0.846709,0.153291,24,1.58,26.0,neg,No TBdx,neg,Cha
1824,f,1,02-399-0214,<1wk,yes,no,yes,yes,yes,no,...,1,0.842927,0.157073,21,1.56,22.0,neg,No TBdx,neg,Cha
1825,m,1,01-399-0973,<1wk,yes,no,yes,no,yes,yes,...,1,0.783166,0.216834,47,1.70,21.0,neg,No TBdx,neg,Kan
1826,m,1,02-399-0711,>4weeks,yes,yes,yes,yes,yes,no,...,1,0.764668,0.235332,23,1.65,20.0,neg,No TBdx,neg,Cha


In [ ]:
count_values = df["ground_truth_tb"].value_counts()
print(count_values)

ground_truth_tb
neg    1623
pos     205
Name: count, dtype: int64


# **Pre-Processing**

## **0_Raw**

In [ ]:
# === Konfigurasi Path ===
BASE_PATH = "/content/drive/My Drive/[3] Model ML TBCare/Dataset Google + CIDRZ Health AI Evaluation Zambia"
METADATA_PATH = os.path.join(BASE_PATH, "Metadata and Codebook/GHAI_Final_Data_2023.csv")

INPUT_FOLDERS = [
    "Chainda South Phone A", "Chainda South Phone B", "Chainda South Phone C",
    "Chawama Phone A", "Chawama Phone B", "Chawama Phone C",
    "Kanyama Phone A", "Kanyama Phone B", "Kanyama Phone C",
    "Audio-Recorder-Chainda-South", "Audio-Recorder-Chawama", "Audio-Recorder-Kanyama"
]

OUTPUT_ROOT = os.path.join(BASE_PATH, "0_Raw")
os.makedirs(os.path.join(OUTPUT_ROOT, "pos"), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_ROOT, "neg"), exist_ok=True)

# === Muat Metadata CSV ===
metadata_df = pd.read_csv(METADATA_PATH)
metadata_df = metadata_df[['sex', 'barcode', 'anon_id', 'ground_truth_tb']]
barcode_to_label = dict(zip(metadata_df['barcode'], metadata_df['ground_truth_tb']))

# === Helper ===
def parse_facility_tool(folder_name):
    if folder_name.startswith("Audio-Recorder-"):
        parts = folder_name.split("-")
        facility = parts[-1]
        if facility == "South":
            facility = "Chainda South"
        return facility, "Audio Recorder"
    else:
        parts = folder_name.split(" Phone ")
        facility = parts[0].strip()
        tool = "Phone " + parts[1].strip()
        return facility, tool

# === Worker ===
def process_file(args):
    folder_in, fname = args
    if not fname.lower().endswith(".wav"):
        return None

    barcode = os.path.splitext(fname)[0]
    tb_label = barcode_to_label.get(barcode, None)
    if tb_label is None:
        return None

    label = "pos" if str(tb_label).lower() in ["positive", "1", "pos"] else "neg"
    src_path = os.path.join(BASE_PATH, folder_in, fname)

    # --- Parse facility & tool ---
    facility, tool = parse_facility_tool(folder_in)

    # --- Mapping tool ke suffix ---
    if "Phone A" in tool:
        suffix = "A"
    elif "Phone B" in tool:
        suffix = "B"
    elif "Phone C" in tool:
        suffix = "C"
    elif "Audio Recorder" in tool:
        suffix = "R"
    else:
        suffix = "X"  # fallback

    # --- Rename file output ---
    out_fname = f"{barcode}-{suffix}.wav"
    dst_path = os.path.join(OUTPUT_ROOT, label, out_fname)

    try:
        shutil.copy2(src_path, dst_path)
        return {
            "barcode": barcode,
            "file_name": out_fname,
            "label": label,
            "sex": metadata_df.loc[metadata_df['barcode'] == barcode, 'sex'].values[0],
            "anon_id": metadata_df.loc[metadata_df['barcode'] == barcode, 'anon_id'].values[0],
            "ground_truth_tb": tb_label,
            "facility_code": facility,
            "tool": tool
        }
    except Exception as e:
        print(f"❌ Error copy {src_path}: {e}")
        return None

# === Main ===
def main():
    # Hitung total file dulu
    all_tasks = []
    folder_to_tasks = {}
    for folder_in in INPUT_FOLDERS:
        full_folder_in_path = os.path.join(BASE_PATH, folder_in)
        if not os.path.exists(full_folder_in_path):
            print(f"⚠️ Skip folder {full_folder_in_path}")
            continue

        tasks = [(folder_in, fname) for fname in os.listdir(full_folder_in_path)]
        if tasks:
            folder_to_tasks[folder_in] = tasks
            all_tasks.extend(tasks)

    print(f"🔍 Total file ditemukan: {len(all_tasks)}\n")

    NUM_PROCESSES = 2
    results = []
    with Pool(processes=NUM_PROCESSES) as pool:
        for folder_in, tasks in folder_to_tasks.items():
            print(f"📂 Proses folder: {folder_in} ({len(tasks)} file)")
            for r in tqdm(pool.imap_unordered(process_file, tasks),
                          total=len(tasks), desc=folder_in, leave=True):
                if r is not None:
                    results.append(r)

    # simpan metadata
    out_meta = pd.DataFrame(results)
    out_meta.to_csv(os.path.join(OUTPUT_ROOT, "metadata_raw.csv"), index=False)
    print(f"\n✅ Selesai! Metadata tersimpan di {OUTPUT_ROOT}/metadata_raw.csv")

if __name__ == "__main__":
    main()

🔍 Total file ditemukan: 1485

📂 Proses folder: Chainda South Phone A (35 file)


Chainda South Phone A: 100%|██████████| 35/35 [01:03<00:00,  1.81s/it]


📂 Proses folder: Chainda South Phone B (35 file)


Chainda South Phone B: 100%|██████████| 35/35 [00:46<00:00,  1.32s/it]


📂 Proses folder: Chainda South Phone C (35 file)


...

📂 Proses folder: Audio-Recorder-Chawama (100 file)


Audio-Recorder-Chawama: 100%|██████████| 100/100 [00:28<00:00,  3.52it/s]


📂 Proses folder: Audio-Recorder-Kanyama (145 file)


Audio-Recorder-Kanyama: 100%|██████████| 145/145 [00:47<00:00,  3.06it/s]


✅ Selesai! Metadata tersimpan di /content/drive/My Drive/[3] Model ML TBCare/Dataset Google + CIDRZ Health AI Evaluation Zambia/0_Raw/metadata_raw.csv


## **1_Segmentasi**

In [ ]:
# === Konfigurasi Path ===
BASE_PATH = "/content/drive/My Drive/[3] Model ML TBCare/Dataset Google + CIDRZ Health AI Evaluation Zambia"
RAW_PATH = os.path.join(BASE_PATH, "0_Raw")
META_RAW_PATH = os.path.join(RAW_PATH, "metadata_raw.csv")
SEG_PATH = os.path.join(BASE_PATH, "1_Segmented")
os.makedirs(os.path.join(SEG_PATH, "pos"), exist_ok=True)
os.makedirs(os.path.join(SEG_PATH, "neg"), exist_ok=True)

META_OUT_PATH = os.path.join(SEG_PATH, "metadata_segmented.csv")

# === Muat Metadata Raw ===
meta_raw = pd.read_csv(META_RAW_PATH)

# === Fungsi VAD ===
def getVad(data, fs):
    winlen, ovrlen, pre_coef, nfilter, nftt = 0.025, 0.01, 0.97, 20, 512
    ftThres = 0.5
    vadThres = 0.4
    opts = 1

    ft, flen, fsh10, nfr10 = speechproc.sflux(data, fs, winlen, ovrlen, nftt)
    pv01 = np.zeros(nfr10)
    pv01[np.less_equal(ft, ftThres)] = 1
    pitch = deepcopy(ft)
    pvblk = speechproc.pitchblockdetect(pv01, pitch, nfr10, opts)

    ENERGYFLOOR = np.exp(-50)
    b = np.array([0.9770, -0.9770])
    a = np.array([1.0000, -0.9540])
    fdata = lfilter(b, a, data, axis=0)

    noise_samp, _, n_noise_samp = speechproc.snre_highenergy(
        fdata, nfr10, flen, fsh10, ENERGYFLOOR, pv01, pvblk
    )
    for j in range(n_noise_samp):
        fdata[round(noise_samp[j, 0]): round(noise_samp[j, 1]) + 1] = 0

    vad_seg = speechproc.snre_vad(
        fdata, nfr10, flen, fsh10, ENERGYFLOOR, pv01, pvblk, vadThres
    )
    return vad_seg

# === Segmentasi ===
def segment_audio(file_path):
    X, sample_rate = librosa.load(file_path, sr=22050, mono=True)
    fvad = getVad(X, sample_rate)

    list_X, temp = [], []
    for i in range(1, len(fvad)):
        if fvad[i - 1] == 1:
            len_sector = math.floor(len(X) / len(fvad))
            start = (i - 1) * len_sector
            for j in range(start, start + len_sector):
                if j < len(X):
                    temp.append(X[j])
        if fvad[i - 1] == 1 and fvad[i] == 0:
            list_X.append(temp)
            temp = []
    return np.array(list_X, dtype=object), sample_rate

# === Worker ===
def process_file(args):
    file_path, barcode, label, row_dict = args
    new_rows = []
    try:
        segments, sr = segment_audio(file_path)
        for i, seg in enumerate(segments):
            if len(seg) == 0:
                continue
            seg = np.asarray(seg, dtype=np.float32)
            out_fname = f"{barcode}_seg{i}.wav"
            out_path = os.path.join(SEG_PATH, label, out_fname)
            sf.write(out_path, seg, sr)

            new_row = row_dict.copy()
            new_row["file_name"] = out_fname
            new_rows.append(new_row)
    except Exception as e:
        print(f"❌ Error {file_path}: {e}")
    return new_rows

# === Main dengan Batch + Checkpoint ===
def main(batch_size=100, n_workers=2):
    # cek checkpoint
    if os.path.exists(META_OUT_PATH):
        done_meta = pd.read_csv(META_OUT_PATH)
        # ambil barcode tanpa suffix _seg untuk checkpoint
        done_files = set(done_meta["file_name"].str.split("_seg").str[0])
        print(f"🔄 Resume mode, sudah ada {len(done_files)} file diproses")
    else:
        done_files = set()

    for label in ["neg", "pos"]:
        folder_in = os.path.join(RAW_PATH, label)
        files = [f for f in os.listdir(folder_in) if f.lower().endswith(".wav")]
        print(f"🔍 {label.upper()} - {len(files)} files")

        # checkpoint → skip yg sudah ada
        files = [f for f in files if os.path.splitext(f)[0] not in done_files]

        for i in range(0, len(files), batch_size):
            batch_files = files[i:i+batch_size]
            print(f"⚡ Batch {i//batch_size+1}/{math.ceil(len(files)/batch_size)} ({len(batch_files)} files)")

            tasks = []
            for fname in batch_files:
                # ambil barcode full tanpa tool suffix
                base_name = os.path.splitext(fname)[0]        # "01-399-1081-A"
                if "-" in base_name:
                    barcode_only = base_name.rsplit("-", 1)[0]  # "01-399-1081"
                else:
                    barcode_only = base_name

                file_path = os.path.join(folder_in, fname)
                row = meta_raw.loc[meta_raw['barcode'] == barcode_only]
                if row.empty:
                    print(f"⚠️ Barcode {barcode_only} tidak ditemukan di metadata")
                    continue
                row_dict = row.iloc[0].to_dict()
                tasks.append((file_path, base_name, label, row_dict))  # gunakan base_name termasuk suffix tool

            new_meta = []
            with Pool(processes=n_workers) as pool:
                for result in tqdm(pool.imap_unordered(process_file, tasks),
                                   total=len(tasks), desc=f"{label}"):
                    new_meta.extend(result)

            # simpan hasil batch (append)
            if new_meta:
                df_out = pd.DataFrame(new_meta)
                header = not os.path.exists(META_OUT_PATH)
                df_out.to_csv(META_OUT_PATH, mode="a", index=False, header=header)

    print(f"✅ Semua selesai! Metadata di {META_OUT_PATH}")


if __name__ == "__main__":
    main(batch_size=100, n_workers=2)

🔍 NEG - 1073 files
⚡ Batch 1/11 (100 files)


neg: 100%|██████████| 100/100 [24:14<00:00, 14.55s/it]

⚡ Batch 2/11 (100 files)



neg: 100%|██████████| 100/100 [23:53<00:00, 14.33s/it]

⚡ Batch 3/11 (100 files)



neg: 100%|██████████| 100/100 [25:40<00:00, 15.40s/it]

⚡ Batch 4/11 (100 files)



neg:  14%|█▍        | 14/100 [04:32<24:06, 16.82s/it]

## **2_Validasi Batuk dan Cleaning**

In [ ]:
# Validasi Batuk

import warnings

# Supress warnings dari librosa/tensorflow
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# === Konfigurasi path ===
BASE_PATH = "/content/drive/My Drive/[3] Model ML TBCare/Dataset Google + CIDRZ Health AI Evaluation Zambia"
SEG_PATH = os.path.join(BASE_PATH, "1_Segmented")
META_SEG_PATH = os.path.join(SEG_PATH, "metadata_segmented.csv")
META_VALIDATED_PATH = os.path.join(SEG_PATH, "metadata_validated.csv")

# === Load YAMNet & class map ===
print("📦 Memuat model YAMNet...")
yamnet_model = hub.load('https://tfhub.dev/google/yamnet/1')
class_names = list(pd.read_csv(os.path.join(BASE_PATH, 'yamnet_class_map.csv'))['display_name'])

# === Baca metadata segmented ===
df = pd.read_csv(META_SEG_PATH)

# Siapkan kolom baru
cough_scores = []
is_coughs = []

# Proses file audio satu per satu
print(f"🔍 Memproses {len(df)} file audio dari folder 1_Segmented...")
for _, row in tqdm(df.iterrows(), total=len(df)):
    label = row['label']      # 'pos' atau 'neg'
    fname = row['file_name']  # nama file segment
    file_path = os.path.join(SEG_PATH, label, fname)

    try:
        waveform, sr = librosa.load(file_path, sr=16000)
        waveform = waveform.astype(np.float32)

        scores, embeddings, spectrogram = yamnet_model(waveform)
        scores_np = scores.numpy()

        cough_index = class_names.index('Cough')
        cough_score = float(np.max(scores_np[:, cough_index]))
        is_cough = 1 if cough_score > 0.5 else 0

    except Exception as e:
        print(f"❌ Gagal memproses {file_path}: {e}")
        cough_score = 0.0
        is_cough = 0

    cough_scores.append(cough_score)
    is_coughs.append(is_cough)

# Tambahkan kolom baru ke dataframe
df["cough_score"] = cough_scores
df["is_cough"] = is_coughs

# Simpan metadata hasil validasi
df.to_csv(META_VALIDATED_PATH, index=False)
print(f"✅ Metadata berhasil disimpan di {META_VALIDATED_PATH}")

In [ ]:
# Cleaning Suara Non Cough

# === Konfigurasi Path ===
BASE_PATH = "/content/drive/My Drive/[3] Model ML TBCare/Dataset Google + CIDRZ Health AI Evaluation Zambia"
SEG_PATH = os.path.join(BASE_PATH, "1_Segmented")
CLEAN_PATH = os.path.join(BASE_PATH, "2_Cleaned")

META_VALID_PATH = os.path.join(SEG_PATH, "metadata_validated.csv")
META_CLEAN_PATH = os.path.join(CLEAN_PATH, "metadata_cleaned.csv")

# === Load metadata validated ===
df = pd.read_csv(META_VALID_PATH)

# Filter hanya segmen yang valid (is_cough == 1)
df_valid = df[df['is_cough'] == 1].copy()
df_valid.reset_index(drop=True, inplace=True)

# Buat folder target
for label in ["pos", "neg"]:
    os.makedirs(os.path.join(CLEAN_PATH, label), exist_ok=True)

# === Worker untuk copy file (dipakai di Pool) ===
def copy_worker(row):
    label = row['label']        # kolom label ('pos'/'neg')
    filename = row['file_name'] # nama file segment
    src_path = os.path.join(SEG_PATH, label, filename)
    dst_path = os.path.join(CLEAN_PATH, label, filename)
    try:
        shutil.copy2(src_path, dst_path)
        return True
    except Exception as e:
        print(f"Gagal menyalin {filename}: {e}")
        return False

# === Main ===
if __name__ == "__main__":
    with Pool(processes=4) as pool:
        results = list(tqdm(pool.imap(copy_worker, [row for _, row in df_valid.iterrows()]),
                            total=len(df_valid),
                            desc="📂 Menyalin audio valid"))

    # Simpan metadata cleaned
    df_valid.to_csv(META_CLEAN_PATH, index=False)
    print(f"✅ Metadata cleaned disimpan ke: {META_CLEAN_PATH}")
    print(f"📊 Total berhasil: {results.count(True)}, gagal: {results.count(False)}")

## **3_Ekstraksi Fitur**

In [ ]:
# === Konfigurasi Path ===
BASE_PATH = "/content/drive/My Drive/[3] Model ML TBCare/Dataset Google + CIDRZ Health AI Evaluation Zambia"
OUTPUT_PATH = "3_Fitur"

labels_map = {'pos': 1, 'neg': 0}

# --- Ekstraksi fitur ---
def extract_features(X, sample_rate):
    stft = np.abs(librosa.stft(X))

    # === Frame-wise features ===
    mfcc = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)              # (40, T)
    chroma = librosa.feature.chroma_stft(S=stft, sr=sample_rate)             # (12, T)
    mel = librosa.feature.melspectrogram(y=X, sr=sample_rate)                # (128, T)
    contrast = librosa.feature.spectral_contrast(S=stft, sr=sample_rate)     # (7, T)
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate)  # (6, T)
    centroid = librosa.feature.spectral_centroid(y=X, sr=sample_rate, n_fft=275)     # (1, T)
    bandwidth = librosa.feature.spectral_bandwidth(y=X, sr=sample_rate, n_fft=275)   # (1, T)
    flatness = librosa.feature.spectral_flatness(y=X, n_fft=275)             # (1, T)
    rolloff = librosa.feature.spectral_rolloff(y=X, sr=sample_rate, n_fft=275)       # (1, T)

    # === Rangkuman statistik ===
    def summarize(feat):
        return np.concatenate([np.mean(feat, axis=1), np.std(feat, axis=1)])

    return {
        "seq": {
            "mfcc": mfcc,
            "chroma": chroma,
            "mel": mel,
            "contrast": contrast,
            "tonnetz": tonnetz,
            "centroid": centroid,
            "bandwidth": bandwidth,
            "flatness": flatness,
            "rolloff": rolloff,
        },
        "mean": {
            "mfcc": summarize(mfcc),
            "chroma": summarize(chroma),
            "mel": summarize(mel),
            "contrast": summarize(contrast),
            "tonnetz": summarize(tonnetz),
            "centroid": summarize(centroid),
            "bandwidth": summarize(bandwidth),
            "flatness": summarize(flatness),
            "rolloff": summarize(rolloff),
        }
    }

def process_file(args):
    file_path, label, root_dir = args
    try:
        X, sr = librosa.load(file_path, sr=None)
        feats = extract_features(X, sr)
        return {
            "mean": {**feats["mean"], "label": labels_map[label], "source": root_dir},
            "seq": {**feats["seq"], "label": labels_map[label], "source": root_dir}
        }
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# --- Kumpulkan file wav ---
def collect_files():
    all_files = []
    for label in os.listdir(BASE_PATH):
        label_dir = os.path.join(BASE_PATH, label)
        if os.path.isdir(label_dir):
            for file_name in os.listdir(label_dir):
                if file_name.endswith(".wav"):
                    file_path = os.path.join(label_dir, file_name)
                    all_files.append((file_path, label, BASE_PATH))
    return all_files

# --- Main ---
def main():
    all_files = collect_files()
    results = []

    with Pool(processes=10) as pool:
        for res in tqdm(pool.imap_unordered(process_file, all_files),
                        total=len(all_files),
                        desc="Extracting features",
                        unit="file"):
            if res is not None:
                results.append(res)

    # --- Simpan hasil ---
    for version in ["mean", "seq"]:
        save_dir = os.path.join(OUTPUT_PATH, version)
        os.makedirs(save_dir, exist_ok=True)

        feature_keys = ['mfcc','chroma','mel','contrast','tonnetz','centroid','bandwidth','flatness','rolloff']
        feature_arrays = {key: [r[version][key] for r in results] for key in feature_keys}
        y_all = [r[version]['label'] for r in results]
        sources = [r[version]['source'] for r in results]

        # Save ke npy
        for key in feature_keys:
            np.save(
                os.path.join(save_dir, f"X_{key}.npy"),
                np.array(feature_arrays[key], dtype=object if version=="seq" else float)
            )
        np.save(os.path.join(save_dir, "y.npy"), np.array(y_all))
        np.save(os.path.join(save_dir, "source.npy"), np.array(sources))

        print(f"✅ Fitur {version} disimpan di: {save_dir}")

if __name__ == "__main__":
    main()

# **Analisis Fitur**